In [14]:
import json
import logging
from datetime import date, datetime
import os

import boto3
from botocore.exceptions import ClientError

In [15]:
AWS_REGION = 'us-east-1'
AWS_PROFILE = 'localstack'
ENDPOINT_URL = 'http://localhost:4566/'

In [16]:
boto3.setup_default_session(profile_name=AWS_PROFILE)
localstack_client = boto3.client("dynamodb", region_name=AWS_REGION, endpoint_url=ENDPOINT_URL)
localstack_resource = boto3.resource("dynamodb", region_name=AWS_REGION, endpoint_url=ENDPOINT_URL)

In [17]:
def json_datetime_serializer(obj):
    if isinstance(obj, (datetime, date)):
        return obj.isoformat()
    raise TypeError("Type %s not serializable" % type(obj))

### Creating a new Table

In [18]:
table_name = 'test_table'

In [21]:
try:
    response = localstack_client.create_table(
        TableName=table_name,
        KeySchema=[
            {
                'AttributeName': 'Name',
                'KeyType': 'HASH' # partition key
            },
            {
                'AttributeName': 'Email',
                'KeyType': 'RANGE' # sort key 
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'Name',
                'AttributeType': 'S'
            },
            {
                'AttributeName': 'Email',
                'AttributeType': 'S'
            }
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 1,
            'WriteCapacityUnits': 1
        },
        Tags=[
            {
                'Key': 'Name',
                'Value': 'dynamodb-table'
            }
        ])
except:
    print('Table already exists')
    

Table already exists


### Adding records to the Table

In [22]:
table = localstack_resource.Table(table_name)
response = table.put_item(
    Item={
        'Name': 'Nikhil',
        'Email': 'nsc@abc.edu'
    }
)

### Reading data from the Table

In [23]:
table = localstack_resource.Table(table_name)
response = table.get_item(
    Key={
        'Name': 'Nikhil',
        'Email': 'nsc@abc.edu'
    }
)